In [13]:
!pip install numerapi
!pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 13.0 MB/s eta 0:00:0000:0100:01


In [1]:
import matplotlib
import numpy as np
import pandas as pd
from numerapi import NumerAPI
import random
import matplotlib.pyplot as plt
import json

In [11]:
napi = NumerAPI()
current_round = napi.get_current_round()
napi.download_dataset("v4.1/train.parquet", "/Users/augustincablant/Desktop/NUMERAI/train.parquet") 
napi.download_dataset("v4.1/validation.parquet", "/Users/augustincablant/Desktop/NUMERAI/validation.parquet")
napi.download_dataset("v4.1/live.parquet", "/Users/augustincablant/Desktop/NUMERAI/live.parquet")
napi.download_dataset("v4.1/features.json", "/Users/augustincablant/Desktop/NUMERAI/features.json")

2023-08-06 20:15:03,675 INFO numerapi.utils: starting download
/Users/augustincablant/Desktop/NUMERAI/train.parquet: 1.45GB [04:20, 5.57MB/s]                            
2023-08-06 20:19:25,926 INFO numerapi.utils: starting download
/Users/augustincablant/Desktop/NUMERAI/validation.parquet: 1.60GB [04:41, 5.68MB/s]                            
2023-08-06 20:24:09,463 INFO numerapi.utils: starting download
/Users/augustincablant/Desktop/NUMERAI/live.parquet: 4.47MB [00:01, 2.67MB/s]                            
2023-08-06 20:24:12,660 INFO numerapi.utils: starting download
/Users/augustincablant/Desktop/NUMERAI/features.json: 703kB [00:00, 817kB/s]                            


'/Users/augustincablant/Desktop/NUMERAI/features.json'

In [2]:
train_set = pd.read_parquet("/Users/augustincablant/Desktop/NUMERAI/train.parquet")

In [18]:
print("Shape of train_set is :", train_set.shape)
print("Columns are :", train_set.columns)

Shape of train_set is : (2420521, 1625)
Columns are : Index(['era', 'data_type', 'feature_honoured_observational_balaamite',
       'feature_polaroid_vadose_quinze', 'feature_untidy_withdrawn_bargeman',
       'feature_genuine_kyphotic_trehala',
       'feature_unenthralled_sportful_schoolhouse',
       'feature_divulsive_explanatory_ideologue',
       'feature_ichthyotic_roofed_yeshiva',
       'feature_waggly_outlandish_carbonisation',
       ...
       'target_thomas_v4_20', 'target_thomas_v4_60', 'target_cyrus_v4_20',
       'target_cyrus_v4_60', 'target_caroline_v4_20', 'target_caroline_v4_60',
       'target_sam_v4_20', 'target_sam_v4_60', 'target_xerxes_v4_20',
       'target_xerxes_v4_60'],
      dtype='object', length=1625)


In [12]:
with open('/Users/augustincablant/Desktop/NUMERAI/features.json', 'r') as json_file:
    data = json.load(json_file)


with open("/Users/augustincablant/Desktop/NUMERAI/features.json", "r") as f:
    feature_metadata = json.load(f)
# features = list(feature_metadata["feature_stats"].keys()) # get all the features
# features = feature_metadata["feature_sets"]["small"] # get the small feature set
features = feature_metadata["feature_sets"]["medium"] # get the medium feature set
# read in just those features along with era and target columns
col_target = []
for name in list(train_set.columns): 
    if name[0:6]=='target':
        col_target.append(name)

read_columns = features + ['era', 'data_type'] + col_target

len(features)

641

In [34]:
# Metric evaluation 

# The models should be scored based on the rank-correlation (spearman) with the target
def numerai_score(y_true, y_pred):
    rank_pred = y_pred.groupby(eras).apply(lambda x: x.rank(pct=True, method="first"))
    return np.corrcoef(y_true, rank_pred)[0,1]

# It can also be convenient while working to evaluate based on the regular (pearson) correlation
def correlation_score(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0,1]

In [13]:
all_feature_corrs = train_set.groupby('era').apply(
    lambda era: era[features].corrwith(era['target']))

: 

: 

In [10]:
riskiest_features = get_biggest_change_features(all_feature_corrs, 50)

['target',
 'target_nomi_v4_20',
 'target_nomi_v4_60',
 'target_tyler_v4_20',
 'target_tyler_v4_60',
 'target_victor_v4_20',
 'target_victor_v4_60',
 'target_ralph_v4_20',
 'target_ralph_v4_60',
 'target_waldo_v4_20',
 'target_waldo_v4_60',
 'target_jerome_v4_20',
 'target_jerome_v4_60',
 'target_janet_v4_20',
 'target_janet_v4_60',
 'target_ben_v4_20',
 'target_ben_v4_60',
 'target_alan_v4_20',
 'target_alan_v4_60',
 'target_paul_v4_20',
 'target_paul_v4_60',
 'target_george_v4_20',
 'target_george_v4_60',
 'target_william_v4_20',
 'target_william_v4_60',
 'target_arthur_v4_20',
 'target_arthur_v4_60',
 'target_thomas_v4_20',
 'target_thomas_v4_60',
 'target_cyrus_v4_20',
 'target_cyrus_v4_60',
 'target_caroline_v4_20',
 'target_caroline_v4_60',
 'target_sam_v4_20',
 'target_sam_v4_60',
 'target_xerxes_v4_20',
 'target_xerxes_v4_60']